# DCGAN TUTORIAL

## Generative Adversarial Networks

### What is a GAN?

GAN은 학습 데이터의 분포를 학습하는 딥러닝 모델입니다. 따라서 이를 통해 같은 분포를 가진 새로운 데이터를 생성할 수 있습니다.
GAN은 generator와 discriminator로 이루어져 있습니다.
generator는 학습한 이미지처럼 진짜같은 '가짜' 이미지를 생성하는 일을 합니다. discriminator는 입력받은 데이터가 진짜 데이터인지 
generator가 생성한 가짜 데이터인지를 구별합니다. 
학습하는 동안 generator는 더욱 진짜같은 데이터를 생성하여 discriminator를 속이기 위해 발전합니다.
반면 discriminator는 학습하는 동안 더욱 가짜 데이터와 진짜 데이터를 잘 구별할 수 있도록 성장합니다.
이러한 둘의 게임은 generator가 완벽한 가짜를 만들어내고 discriminator의 구별하는 능력이 최상에 도달하였을 때 균형에 도달하게 됩니다.
이때 discriminator는 50%의 확률로 데이터를 추측하게 됩니다. 

discriminator의 정의부터 시작해보겠습니다. x를 이미지를 의미하는 데이터라고 할 때, D(x)는 x가 진짜인지 아닌지의 확률을 출력하는 discriminator network입니다.
이미지를 예시로 하면 D(x)의 인풋은 CHW(channel, height, width) 3*64*64 크기의 이미지입니다. 
만약 입력받은 이미지가 진짜 이미지라면 높은 값을 출력하고, 반대로 generator에서 만든 가짜 이미지라면 낮은 값을 출력하게 됩니다. 
즉 D(x)는 전통적인 binary classfier라고 볼 수 있습니다. 

이번에는 generator에 대해서 알아보도록 하겠습니다. z를 표준정규분포에서 뽑은 잠재공간벡터라고 할 때 G(z)는 latent vector z를 data space로 매핑하는 generator 함수입니다.
G의 목표는 학습 데이터의 분포를 추정하여 추정된 분포에서 뽑아낸 것 같은 가짜 데이터를 생성해내는 것입니다. 
따라서, D(G(z))는 G가 생성한 이미지가 진짜일 확률을 출력하는 함수가 됩니다. 
Goodfellow의 논문에 따르면 D와 G는 minimax게임을 하여 D는 가짜와 진짜를 잘 구별하는 척도인 log(D(x))를 최대화하고
G는 log(1-D(G(z)))를 최소화하는 목적을 가지게 됩니다. 
이론적으로 이러한 minimax게임의 solution은 pg = pdata로서, discriminator가 50대 50확률 랜덤으로 인풋을 구별하는 것입니다. 

### What is DCGAN?

DCGAN은 위에서 언급하였던 GAN의 확장 개념으로서, 합성곱층(convolutional)을 discriminator와 generator에 적용한 모델입니다.
discriminator는 convolutional, batch norm, LeakyReLU 활성화로 구성되어 있습니다. 인풋은 3*64*64의 이미지이며 이것의 아웃풋은 입력받은 이미지가 진짜일 확률입니다.
generator는 convolutional-transpose layers, batch norm, ReLU로 구성되어 있습니다. 인풋은 표준정규분포에서 뽑은 잠재 벡터 z, 아웃풋은 3*64*64 크기의 RGB 이미지입니다.
strided conv-transpose layer는 잠재벡터를 이미지와 같은 크기로 변형시켜 주는 역할을 하게 됩니다. 

In [1]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


## Inputs

* **dataroot** - dataset 폴더의 루트 경로 
* **workers** - DataLoader에서 데이터를 로드하는 worker thread의 수
* **batch_size** - 학습에 사용하는 배치의 사이즈. DCGAN논문에서는 128을 사용
* **image_size** - 학습에 사용하는 이미지 공간 사이즈. 이번 구현에서는 64*64를 사용. 만약 다른 크기를 사용하고 싶다면 D와 G의 구조를 바꿔주어야함
* **nc** - 입력 이미지의 채널 개수 이번에는 3 사용
* **nz** - 잠재 벡터의 길이
* **ngf** - generator를 통과하는 피쳐 맵의 깊이
* **ndf** - discriminator를 통과하는 피쳐 맵의 깊이
* **num_epochs** - 학습 반복 횟수. 값을 크게 하면 좋은 결과를 얻겠지만 그만큼 시간도 오래 걸림.
* **lr** - learning rate. DCGAN 논문에서는 0.0002
* **beta1** - Adam optimizer의 하이퍼파라미터. 논문에서는 0.5 사용
* **ngpu** - 사용 가능한 GPU 개수


In [3]:
# Root directory for dataset
dataroot = "data/celeba"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

## Data

이번 예제에서는 Celeb-A Faces 데이터셋을 사용합니다. 데이터셋은 img_align_celeba.zip 형태로 다운로드되며, celeba라는 디렉토리를 생성하여 압축를 해제하세요. 생성한 폴더의 경로를 dataroot 변수에 넣어주세요. 디렉토리 구조는 아래와 같게 됩니다:

In [4]:
/path/to/celeba

NameError: name 'path' is not defined

데이터셋의 루트 폴더의 하위 디렉토리가 ImageFolder 데이터셋 클래스에 필요합니다. 
데이터셋을 생성하고, dataloader를 생성하여 장치에서 실행하여 학습한 데이터를 시각화해봅니다.

In [11]:
# We can use an image folder dataset the way we have it setup.
# Create the dataset
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Plot some training images
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'data/celeba'

## implementation

인풋 파라미터와 데이터셋이 준비되면 이제 구현이 가능합니다! 먼저 가중치 초기화부터 시작하고 generator, discriminator, loss function, training loop를 다루도록 하겠습니다.

### Weight Initialization

DCGAN 논문에서 저자는 모델의 가중치는 평균이 0, 분산이 0.02인 정규 분포로부터 랜덤하게 초기화되어한다고 주장합니다. 
weight_init 함수가 모든 convolutional, convolutional-transpose, 그리고 batch normalization layer를 조건 충족되게 재초기화할 것 입니다.
이 함수는 초기화가 진행된 후에 실행됩니다. 

In [6]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

### Generator

generator G 는 잠재 공간 벡터 z를 데이터 공간에 매핑하는 역할을 합니다. 이번 구현에서 우리 데이터는 이미지이고, z를 데이터 공간으로 변환한다는 것은 결론적으로는 학습 데이터와 같은 크기 (3*64*64)의 RGB 이미지를 만든다는 것을 의미합니다.
실질적으로 이러한 작업은 각각 2d batch norm layer와 relu 활성화와 짝을 이룬 2차원 convolutional transpose layers을 통해 이루어집니다. 
generator의 결과물은 tanh 활성화를 거쳐 인풋 이미지를 -1, 1 사이의 값을 갖게 합니다. 
batch norm 함수가 conv-transpose layers 바로 뒤에 존재하는데 이것이 DCGAN 논문에서 언급한 중요한 점입니다. 이 layer들은 학습하는 동안 gradient가 잘 전파되도록 도와줍니다. 
DCGAN 논문의 generator의 구조 이미지는 아래와 같습니다. 

인풋 섹션(nz, ngf, nc)가 generator architecture에서 어떤 영향을 끼치는지 보도록 하겠습니다.
nz는 z 인풋 벡터의 크기이고, ngf는 generator의 피쳐맵의 깊이 관련되고, nc는 결과 이미지의 채널과 관련됩니다.
아래는 generator의 코드입니다. 

In [9]:
# Generator Code

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

이제 generator 객체를 생성하고 weights_init 함수를 실행하겠습니다. 
출력된 모델 값을 관찰하여 generator 객체가 어떤 구조를 가지고 있는지 확인해보세요.

In [10]:
netG = Generator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netG.apply(weights_init)

# Print the model
print(netG)

NameError: name 'device' is not defined

### Discriminator

discriminator D는 위에서 언급한대로 이미지가 가짜인지 진짜인지 판별하는 역할을 합니다.
여기서, D는 3*64*64 인풋 이미지를 받고, Conv2d, BatchNorm2d, 그리고 LeakyReLU 레이어를 거쳐 마지막에 sigmoid 활성화를 거쳐 최종적으로 확률을 출력합니다. 
이 구조는 좀 더 많은 레이어를 쌓는 것이 가능하나, DCGAN 논문에서는 strided convolution을 사용하는 것이 네트워크가 스스로 pooling 함수를 학습하기 때문에 pooling을 통한 downsampling보다 실질적으로는 더 좋다고 말하고 있습니다.
또한 batch norm 과 leaky relu 함수는 G와 D에서 gradient를 잘 전달하게 하여 학습을 더 잘 진행하도록 도와줍니다. 

In [13]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

generator 때와 동일하게 weights_init 함수를 discriminator에 적용하고 모델의 구조를 출력해봅니다,

In [ ]:
# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netD.apply(weights_init)

# Print the model
print(netD)


### Loss Functions and Optimizers 

D와 G의 세팅이 완료되면, 이제 loss function과 optimizers를 알아보도록 하겠습니다.
이번 예제에서는 Binary Cross Entropy loss를 사용합니다. 
이 함수는 두 로그함수로 이루어져 있음을 주목하세요. BCE 방정식에서 어느 부분이 y 인풋과 사용되는지를 구별할 수 있습니다. 
여기서 진짜는 1, 가짜는 0으로 구별합니다. 이러한 라벨링은 D와 G의 LOSS를 계산할 때 사용되며, 원래 GAN논문에서부터 언급되었던 관습같은 것 입니다. 
드디어 두 개의 다른 optimizer 세팅하였습니다. 하나는 D, 다른 하나는 G의 것 입니다.
DCGAN 논문에서 언급되기를 두 optimizer 모두 Adam을 사용하고, learning rate = 0.00002, Beta1 = 0.5으로 설정하였습니다. generator의 학습 과정을 추적해보면 가우시안으로부터 추출된 고정된 배치의 잠재 벡터를 생성하는 것을 알 수 있습니다. training loop에서 주기적으로 G에 이 fixed_noise를 입력값으로 넣어줍니다. 반복이 계속되면 노이즈로부터 이미지가 생성되는 것을 볼 수 있습니다. 

In [ ]:
# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

### Training

드디어 GAN 프레임워크의 준비를 마쳤으니 훈련을 시작할 수 있습니다. 주의할 점은 GAN을 학습할 때 하이퍼파라미터를 잘못 세팅하면 mode collapse가 일어날 수 있습니다. 
여기서 학습 단계를 두 단계로 나누어 진행합니다. part1에서 Discriminator를 업데이터하며 part2에서 Generator를 업데이트 합니다. 

#### Part 1 - Train the Discriminator 

discriminator는 진짜와 가짜를 구별하는 확률을 최대화하는 것이 목적입니다. 즉 log(D(x))+log(1-D(G(z)))를 최대화하는 것이 목적입니다,
먼저, 학습 세트에서 진짜 샘플을 뽑아서 D를 통과시켜 log(D(x))를 계산합니다. 그리고 역전파를 계산합니다. 
다음으로, 현재 있는 generator로부터 가짜 샘플을 생성하여 D에 입력값으로 넣어줍니다. 그리고 log(1-D(G(z)))를 계산하여 역시 역전파도 계산합니다.
이제 gradient가 진짜와 가짜에 대해서 모두 쌓여있으니 discriminator의 optimizer를 호출합니다.

#### Part 2 - Train the Generator

이번에는 Generator를 log(1-D(G(z)))를 최소화하는 방향으로 학습을 진행합니다. 즉 log(D(G(Z)))를 최대화합니다. 
코드에서 우리는 생성자의 아웃풋을 part1의 discriminator로 분류를 하여, G의 gradient를 통해 역전파를 계산합니다. 
그리고 G 의 파라미터 값을 optimizer step에 따라 업데이트를 진행합니다. 
각 epoch의 끝에 fixed_noise를 generator에 주입하여 G의 학습 과정을 살펴봅니다.
학습 과정에서 출력되는 통계는 아래와 같습니다.
* **Loss_D** - 모든 진짜와 가짜 배치에 대한 Discriminator의 loss 값의 합
* **Loss_G** - log(D(G(z)))로 계산된 generator loss
* **D(x)** - 모든 진짜 배치에 대한 discriminator의 평균 아웃풋. 이론적으로 1에서 시작하여 G가 학습함에 따라 0.5에 수렴
* **D(G(z))** - 모든 가짜 배치에 대한 discriminator의 평균. 0에서 시작하여 G가 학습함에따라 0.5에 수렴.


In [ ]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

### Results

마지막으로 한 일들을 체크해보겠습니다. 세가지 다른 결과를 볼 것 입니다.
첫번째는 학습하는 동안 D와 G의 loss의 변화량입니다.
두번째는 매 epoch마다 fixed_noise에 대한 G의 아웃풋의 변화를 시각화입니다.
세번째는 진짜 데이터와 가짜 데이터의 비교입니다.

#### Loss versus training iteration

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

#### Visualization of G's progression 

#%%capture
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

#### Real Images vs. Fake Images

In [ ]:
# Grab a batch of real images from the dataloader
real_batch = next(iter(dataloader))

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()